In [0]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()
import ee
ee.Authenticate()
ee.Initialize()

In [0]:
def add_ndvi(img):
  nir = img.select('B8')
  red = img.select('B4')
  ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
  return img.addBands(ndvi)

def select_bands(img):
  return img.expression('b("B6", "B5", "B4")')

def filterCloudSentinel2(img):
  #
  #Bitmask for QA60
    #Bit 10: Opaque clouds
        #0: No opaque clouds
        #1: Opaque clouds present
    #Bit 11: Cirrus clouds
        #0: No cirrus clouds
        #1: Cirrus clouds present
  #
  quality = img.select('QA60').int()
  cloudBit = ee.Number(1024)    # ee.Number(2).pow(10);
  cirrusBit = ee.Number(2048)  # ee.Number(2).pow(11)
  
  cloudFree = quality.bitwiseAnd(cloudBit).eq(0)
  cirrusFree = quality.bitwiseAnd(cirrusBit).eq(0)
  clear = cloudFree.bitwiseAnd(cirrusFree)
  
  return img.updateMask(clear)

In [0]:
roi = ee.FeatureCollection('users/baskinrobbinsman/IFL').first().geometry()

In [0]:
Sent2 = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate('2018-6-01','2018-9-30').map(filterCloudSentinel2)
medS2 = Sent2.median().clip(roi).select(["B12","B8","B4"])

Sent2020 = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate('2019-6-01','2019-9-30').map(filterCloudSentinel2)
medS2020 = Sent2020.median().clip(roi).select(["B12","B8","B4"])

In [0]:
S2_diff = medS2.subtract(medS2020)
S2_diff = S2_diff.divide(S2_diff.reduce(ee.Reducer.max()))

In [0]:
def create_evi(img):
  evi = img.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': img.select('B8'),
        'RED': img.select('B4'),
        'BLUE': img.select('B2')}).rename('EVI')
  return evi
evi_1 = create_evi(medS2)
evi_2 = create_evi(medS2020)
evi_diff = evi_1.subtract(evi_2)
ndvi_1 = add_ndvi(medS2).select("NDVI")
ndvi_2 = add_ndvi(medS2020).select("NDVI")
ndvi_diff = ndvi_1.subtract(ndvi_2)

Кластеризация (не пригодилась)

In [0]:
train_2018 = medS2.select(["B12","B8","B4"]).sample(**{
  'region': roi,
  'scale': 20,
  'numPixels': 5000})

clusterer = ee.Clusterer.wekaKMeans(10).train(train_2018)
res_2018 = medS2.select(["B12","B8","B4"]).cluster(clusterer)
res_2020 = medS2020.select(["B12","B8","B4"]).cluster(clusterer)
task = ee.batch.Export.image.toDrive(res_2018, folder='AKS/clusters')
task.start()

In [0]:
import folium
mapid = res_2018.randomVisualizer().getMapId()
mymap = folium.Map(location=[*roi.centroid().getInfo()['coordinates'][::-1]])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='clusters 2018',
  ).add_to(mymap)
mapid = res_2020.randomVisualizer().getMapId()
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='clusters 2020',
  ).add_to(mymap)
mymap.add_child(folium.LayerControl())
mymap

Эсперименты с вегетативными индексами

In [0]:
evi_diff_0 = evi_1.expression('(OLD > 0) & (NEW <= 0)', {'OLD': evi_1, 'NEW': evi_2})
evi_diff_1 = evi_1.expression('(OLD > 0.3) & (NEW <= 0)', {'OLD': evi_1, 'NEW': evi_2})
diff_diff = evi_diff_0.subtract(evi_diff_1)
diff_m = diff_diff.eq(ee.Number(1))
diff_d = diff_m.updateMask(diff_m)
ndvi_diff = ndvi_1.expression('(OLD > 0.5) & (NEW <= 0.5)', {'OLD': ndvi_1, 'NEW': ndvi_2}) # слишком большой порог
target = evi_diff_1.eq(ee.Number(1))
diff = target.updateMask(target)
nd = ndvi_diff.eq(ee.Number(1))
ndvi_d = nd.updateMask(nd)
B4_diff = S2_diff.select("B8").gte(ee.Number(2.5))
B4_d = B4_diff.updateMask(B4_diff)

In [0]:
# clusters_diff = res_2018.expression('OLD != NEW', {'OLD': res_2018, 'NEW': res_2020})
import folium
mapid = medS2020.getMapId({'bands': ['B12', 'B8', 'B4'], 'min': 500, 'max': 5000})
mymap = folium.Map(location=[*roi.centroid().getInfo()['coordinates'][::-1]])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='diff',
  ).add_to(mymap)

mapid = ndvi_d.getMapId({'palette': ['red']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='diff B4',
  ).add_to(mymap)
mymap.add_child(folium.LayerControl())
mymap

In [0]:
polygons = ndvi_d.reduceToVectors(**{
  'geometry': roi,
  'crs': Sent2.first().select('B4').projection(),
  'scale': 20,
  'geometryType': 'polygon',
  'eightConnected': False,
  'labelProperty': 'raster_value',
  'reducer': ee.Reducer.countEvery(),
  'maxPixels': 100000000})

In [0]:
mapid = ee.FeatureCollection('users/baskinrobbinsman/evi_diff').getMapId()
mymap = folium.Map(location=[*roi.centroid().getInfo()['coordinates'][::-1]])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='diff_poly',
  ).add_to(mymap)

mapid = ee.Feature(ee.FeatureCollection('users/baskinrobbinsman/IFL').first()).getMapId()
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='bounds',
  ).add_to(mymap)

mapid = diff.getMapId({'palette': ['red']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='diff',
  ).add_to(mymap)

mymap.add_child(folium.LayerControl())
mymap

In [0]:
task = ee.batch.Export.table.toDrive(polygons, folder='ndvi_diff', fileFormat='shp', fileNamePrefix='ndvi')
task.start()

In [0]:
task = ee.batch.Export.image.toDrive(medS2, scale=20,
                                     folder='composites', description='comp1', 
                                     fileNamePrefix='S2_2018', maxPixels=10000000000000)
task.start()

In [0]:
task = ee.batch.Export.image.toDrive(medS2020, scale=20,
                                     folder='composites', description='comp2', 
                                     fileNamePrefix='S2_2020', maxPixels=10000000000000)
task.start()

Деление композитов на мелкие кусочки ради нормальной отрисовки в QGIS

In [0]:
import os, gdal
 
in_path = '/content/drive/My Drive/composites/'
input_names = os.listdir(in_path)
 
out_path = '/content/drive/My Drive/tiles/'
output_filename = 'tile_'

tile_size_x = 2000
tile_size_y = 2000

In [0]:
os.mkdir(out_path)

In [0]:
#print(band.XSize, band.YSize)
os.listdir(out_path)

['S2_2018_1_0_8000.tif',
 'S2_2018_1_0_10000.tif',
 'S2_2018_2_0_0.tif',
 'S2_2018_2_0_2000.tif',
 'S2_2018_2_0_4000.tif',
 'S2_2018_2_2000_4000.tif',
 'S2_2018_2_2000_6000.tif',
 'S2_2018_2_2000_8000.tif',
 'S2_2018_2_2000_10000.tif',
 'S2_2018_2_4000_0.tif',
 'S2_2018_2_8000_8000.tif',
 'S2_2018_2_10000_0.tif',
 'S2_2018_2_10000_4000.tif',
 'S2_2018_2_10000_6000.tif',
 'S2_2018_2_10000_8000.tif',
 'S2_2020_4_10000_10000.tif',
 'S2_2018_1_0_0.tif',
 'S2_2018_1_0_2000.tif',
 'S2_2018_1_0_4000.tif',
 'S2_2018_1_0_6000.tif',
 'S2_2018_2_0_6000.tif',
 'S2_2018_2_0_8000.tif',
 'S2_2018_2_0_10000.tif',
 'S2_2018_2_2000_0.tif',
 'S2_2018_2_2000_2000.tif',
 'S2_2018_2_4000_2000.tif',
 'S2_2018_2_4000_4000.tif',
 'S2_2018_2_4000_6000.tif',
 'S2_2018_2_4000_8000.tif',
 'S2_2018_2_4000_10000.tif',
 'S2_2018_2_6000_0.tif',
 'S2_2018_2_6000_2000.tif',
 'S2_2018_2_6000_4000.tif',
 'S2_2018_2_6000_6000.tif',
 'S2_2018_2_6000_8000.tif',
 'S2_2018_2_6000_10000.tif',
 'S2_2018_2_8000_0.tif',
 'S2_2018_

In [0]:
from tqdm import tqdm

for k, name in tqdm(enumerate(input_names)):
  ds = gdal.Open(in_path + name)
  band = ds.GetRasterBand(1)

  xsize = band.XSize
  ysize = band.YSize
  
  for i in range(0, xsize, tile_size_x):
      for j in range(0, ysize, tile_size_y):
          com_string = f'gdal_translate -of GTIFF -srcwin {i}, {j}, {tile_size_x}, {tile_size_y} "{in_path + name}" "{out_path + name.split("-")[0]}_{k % 2 + 1}_{i}_{j}.tif"'
          os.system(com_string)
          #print(com_string)

4it [02:15, 33.81s/it]


Переименовывание файлов для сокомандников

In [0]:
for fname in os.listdir(out_path):
  os.rename(out_path + fname, out_path + fname.replace('ГРИША_', '').replace('ПОЛИНА_', '').replace('ЛИЗА_', ''))

In [0]:
files = list(filter(lambda x: x.startswith('S2_2018'), os.listdir(out_path)))
for i, who in enumerate(['ГРИША_', 'ПОЛИНА_', 'ЛИЗА_']):
  for fname in files[i * 16: (i + 1) * 16]:
    os.rename(out_path + fname, out_path + who + fname)
    nf = 'S2_2020_' + '_'.join(fname.split('_')[2:])
    os.rename(out_path + nf, out_path + who + nf)
os.listdir(out_path)